# **RED NEURONAL CLAVES CURVAS ELIPTICAS**

In [ ]:
!git clone https://github.com/lc6chang/ecc-pycrypto.git

In [ ]:
!pip install /content/ecc-pycrypto/.

In [ ]:
from keras.models import Sequential,Model
from keras.layers import SimpleRNN, Input, Dense
import numpy as np
import matplotlib.pyplot as plt 
from keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import string
import random
import hashlib

from ecc.curve import Curve25519
from ecc.key import gen_keypair
from ecc.cipher import ElGamal

# Parametros configuración inicial
word_size = 6 # tamaño cadenas
hash = "sha1" # "sha256"

# Declaración de parámetros
nn = [256*2,300,500,300,256]
X_train = []
Y_train = []
X_test = []
Y_test = []
X = []
Y = []

pub_train, pri_train = generate_keys(10000)
pub_test, pri_test = generate_keys(1000)

In [ ]:
def crearModelo(loss='poisson',activation='relu', unit1=500, unit2=750,optimizer='RMSprop'):

  # Definir modelo    
  model = Sequential()

  # Capas
  model.add(Dense(nn[4], input_dim=512))
  model.add(Dense(unit1, activation='relu'))
  model.add(Dense(unit2, activation='relu'))
  model.add(Dense(unit1, activation='relu'))
  model.add(Dense(nn[4], activation=activation))

  # Compilar modelo
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

In [ ]:
model = crearModelo()

snn = model.fit(pub_train, pri_train, epochs=120, batch_size=400)

scores = model.evaluate(pub_test, pri_test)   
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


pred = model.predict(pub_test, batch_size=400, verbose=1)  
print(pred) 

In [ ]:
def generate_keys (number_keys):
  pub_keys = np.zeros((number_keys,512))
  pri_keys = np.zeros((number_keys,256))
  for x in range(number_keys):
    pri, pub = gen_keypair(Curve25519)
    y=255
    while pub.x != 0:
      pub_keys[x][y]= pub.x % 2
      pub.x //= 2
      y-=1
    y=511
    while pub.y != 0:
      pub_keys[x][y]= pub.y % 2
      pub.y //= 2
      y-=1
    y=255
    while pri != 0:
      pri_keys[x][y]= pri % 2
      pri //= 2
      y-=1
  return pub_keys, pri_keys
  